# Customer Personality Analysis - Unsupervised Learning Project

Kaggle Dataset: https://www.kaggle.com/datasets/imakash3011/customer-personality-analysis
        
Kaggle Notebook: https://www.kaggle.com/code/robinlutter/customer-personality-analysis-pca-and-clustering

GitHub: https://github.com/wr0b1n/MSDS-5510-FinalProject

# 1. Project Topic and Goal

In this project we want to build an unsupervised machine learning model that is able to perform a clustering for a  segmentation task on customer data. To achieve this, we are working with a public dataset from Kaggle that is not part of a competition which means there is no pre-specified evaluation metric. The goal is to compare different clustering algorithms with each other and see how well they perform. Besides typical clustering methods like K-Means we may need to perform a PCA to reduce the amount of dimensions in our dataset. 

Why would we even want to perform such a customer segmentation from a business point of view?

The reason is that this analysis empowers businesses to make the right product modifications or marketing strategies that align precisely with the preferences of specific customer segments. For example, rather than investing resources in marketing a new product to every customer available, the company can identify the most receptive customer segment and focus its marketing efforts only on that particular group. This targeted approach enhances efficiency, saves costs, and increases the likelihood of successful product adoption. This means we can understand our customer's needs better, distinct them by their buying behavior and ultimately adapt our marketing strategy and product development accordingly.

During this project we aim for answering the following business question. Can we segment our customers by looking at their income and money spent on our products? This will hopefully help us to target appropriate marketing strategies at certain groups of customers and finally increase our revenue. This way we won't waste effort in marketing products on people that won't buy them anyway but instead make our marketing strategy much more effective.

# 2. Data

The project is based on the following public Kaggle dataset: https://www.kaggle.com/datasets/imakash3011/customer-personality-analysis

The dataset itself has a CSV format with a size of around 220kB which is really small. This means we don't have to worry processing large amounts of data. The format is tabular with 29 columns and 2240 rows in total.

A first glance at the data shows that we have to work with features both numerical and categorical. We will inspect this in more detail during the EDA part. The features itself can be grouped into different categories:

* People (Year of birth, education level, ...)
* Products (Amount spent on fruits in last 2 years, ...)
* Promotion (Number of purchases made with discount, ...)
* Place (Number of purchases made through website, ...)

# 3. Import Python Libraries

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import calinski_harabasz_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import silhouette_score, make_scorer
from sklearn.metrics import davies_bouldin_score

import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")

# 4. Exploratory Data Analysis

We will start with exploring our data set several built-in dataframe methods and visualizations.

## 4.1 Data Description

In [ ]:
df = pd.read_csv('/kaggle/input/customer-personality-analysis/marketing_campaign.csv', sep='\t')

In [ ]:
df.head()

In [ ]:
df.info()

We are working with a dataset of 2240 rows and 29 columns. We inspect that there are some missing values in the "Income" column. We will take care of these in the next part.

Below we have listed the categorical and numerical features which we will make use of later. 

In [ ]:
categorical_features = ['Education', 'Marital_Status', 'Complain', 'AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'Response']
print(categorical_features)

In [ ]:
numerical_features = [feature for feature in df.columns.tolist() if feature not in categorical_features]
print(numerical_features)

## 4.2 Data Cleaning and Preprocessing

### 4.2.1 Missing Values

First, we check for NA or NULL values using the built-in functions of pandas dataframes (to confirm our inspection of missing values from before).

In [ ]:
df.isna().sum()

In [ ]:
df.isnull().sum()

Just in case, we can also search for empty (or just spaces) string values. 

In [ ]:
np.where(df.applymap(lambda x: str(x).strip() == ''))

The observation shows that there are indeed 24 NA respectively NULL values in the 'Income' column but no empty strings or spaces in any other column. Since income might be an important indicator wether a customer belongs to this or another cluster we should definitely take care of them. There are several commonly used strategies:

* Impute missing values based on their underlying distribution
* Drop rows with empty values
* Use advanced imputation techniques like regression imputation

We will start by looking at the distribution of the valid values.

In [ ]:
def plot_income_dist(df):
    plt.hist(df['Income'], bins=50)
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.title('Histogram of Income')
    plt.show()

In [ ]:
plot_income_dist(df)

From the histogram we can assume that most of the income data follows a normal distribution. But we have a few outliers to to the right side at an income of approximately 150000 and even one extreme outlier at an income of over 600000. Let's define a treshold at 200000 and see how many samples fall below and above it.  

In [ ]:
# calculate percentage of samples above and below the threshold
threshold = 200000
above_threshold = df[df['Income'] > threshold].shape[0]
below_threshold = df[df['Income'] <= threshold].shape[0]
total_samples = above_threshold + below_threshold

percentage_above_threshold = (above_threshold / total_samples) * 100
percentage_below_threshold = (below_threshold / total_samples) * 100

# create pie chart
labels = ['Above Threshold', 'Below Threshold']
sizes = [percentage_above_threshold, percentage_below_threshold]
colors = ['skyblue', 'lightgreen']
explode = (0.2, 0)

plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.2f%%', shadow=True, startangle=140)
plt.axis('equal')
plt.title(f'Pie chart for distribution of Income (Threshold = {threshold})')
plt.show()

Since only around 0.05% of our customers have an income of over 200000 we can safely ignore these by dropping the respective samples. Since there are still a few customers with income of around 150000 we cannot assume normality for the distribution due to the skewness to the right side. But we do not want to drop them since they could be part of an high income cluster or even form its own cluster. As a consequence, we cannot simply impute the missing values by using the mean but rather use the median value. 

In [ ]:
# drop customers with income greater than 200000
mask = df['Income'] > 200000
df = df[~mask]

In [ ]:
# impute median income for missing values
median_income = df['Income'].median()
df['Income'].fillna(median_income, inplace=True)
df.isna().sum()

The output shows that there are no missing values anymore. Again, we can have a look at the histogram to see the underlying distribution in greater resolution containing the imputed values.

In [ ]:
plot_income_dist(df)

### 4.2.3 Inspecting numerical features

Next, we will inspect all the numerical features and check whether there are any other extreme outliers present in the data.

In [ ]:
# make a grid for all the plots
num_plots = len(numerical_features)
num_cols = 4
num_rows = int(num_plots / num_cols)

fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 5*num_rows))
fig.suptitle('Numerical Features Plots', fontsize=16)

# flattens the axs array (otherwise axs[i].hist does not work)
axs = axs.ravel()

# make a plot for each numerical feature
for i, column in enumerate(numerical_features):
    axs[i].hist(df[column], bins=20)
    axs[i].set_title(column)
    axs[i].set_xlabel('Values')
    axs[i].set_ylabel('Frequency')
    
plt.show()

The first thing to notice here is that both the 'Z_CostContact' and 'Z_Revenue' column have the same value over all samples. Unfortunetaly, the dataset provides no explanation what these two columns are about. But since there is always the same value there is no impact on our model building and we can simply drop them.

We can also see that the ID column (by nature) has no significant meaning for our clustering approach. We will drop this one as well.

Many features seem to follow some sort of exponential distribution. For example, looking at the 'MntFishProducts' column that means that most of our customers have spent a pretty small amount of money on fish in last 2 years. This makes sense since fish often times can be quite expensive and less people have enough buying power to purchase large amounts of it.

There are also some features with outliers. For example, the 'NumDealsPurchases' column which represent the number of purchases made with a discount has quite a few samples with higher numbers. But again, this could be of significant meaning to our clutering goal and potentially form its own cluster. Therefore we won't drop those samples.

In [ ]:
# drop irrelevant columns
columns_to_drop = ['ID', 'Z_CostContact', 'Z_Revenue']
df.drop(columns=columns_to_drop, axis=1, inplace=True)

We should also take care of the "Dt_Customer" column which represent the date of the customer's enrollment with the company. This representation will likely cause problems when applying techniques as PCA and model building due to its datatype. Therefore, we will convert this column into a new column that represents the number of days the customer is enrolled with the company which we can represent using an integer.

In [ ]:
# convert to datetime format
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='%d-%m-%Y')
current_date = specified_datetime = datetime.strptime("29-07-2023", "%d-%m-%Y")

# get number of days since enrollment
df['DaysEnrolled'] = (current_date - df['Dt_Customer']).dt.days

# drop original column since it is not needed anymore
df.drop('Dt_Customer', axis=1, inplace=True)

The fact that we use the current date as a reference should not impact model performance since we will standardize our data before applying PCA and model building. The main focus here is to catch the relative durations for each customer to compare them with each other.

Since clustering methods can suffer from high dimensionality we should aim for reducing dimensions as much as possible. For this purpose we will combine all columns related to spending a certain amount on a specific product and create a new column that shows the total amount spent.

In [ ]:
# combine columns and drop original ones
df['TotalSpent'] = df['MntWines'] + df['MntFruits'] + df['MntMeatProducts'] + df['MntFishProducts'] + df['MntSweetProducts'] + df['MntGoldProds']
columns_to_drop = ['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']
df.drop(columns=columns_to_drop, inplace=True)

In [ ]:
df.head()

### 4.2.4 Inspecting categorical features

In this step, we will have a look at the categorical features by creating barplots using seaborn. We will see how the different categories are encoded and if we have to clean things up.

In [ ]:
# make a grid for the plots
num_plots = len(categorical_features)
num_cols = 2
num_rows = (num_plots - 1) // num_cols + 1

fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 5*num_rows))
fig.suptitle('Categorical Features Plots', fontsize=16)

# flattens the axs array (otherwise axs[i].bar does not work)
axs = axs.ravel()

# plot all categorical features
for i, column in enumerate(categorical_features):
    counts = df[column].value_counts()
    axs[i].bar(counts.index, counts.values)
    axs[i].set_title(column)
    axs[i].set_xlabel('Categories')
    axs[i].set_ylabel('Count')
    
# hide unused grid space
for i in range(len(categorical_features), num_rows * num_cols):
    axs[i].axis('off')
    
plt.show()

Except for 'Education' and 'Marital_Status' all categorical features take on only 2 possible values 0 and 1.

Most of our customers are married or at least living together. For the 'Marital_Status' feature we have many different possible values. There are also some categories like 'YOLO' or 'Absurd' that don't make sense and are not explained any futher in the dataset description on Kaggle. We will drop these two since they only represent a very small part of the samples. To reduce model complexity futher we will combine 'Married' and 'Together' to create a new category 'Relationship' and we will also combine 'Single', 'Divorced', 'Widow', and 'Alone' to a new category 'No Relationship'.

For 'Education' it is quite hard to say what 'Graduation', '2n Cycle', and 'Basic' means. I could not find any explanation on Kaggle for this. Since those categories are not negligible due to the corresponding amount of customers we will leave them as they are. 

In [ ]:
# remove samples with 'Absurd' and 'YOLO' marital status
mask1 = df['Marital_Status'] == 'Absurd'
mask2 = df['Marital_Status'] == 'YOLO'
df = df[~mask1]
df = df[~mask2]

# combine categories of marital status
category_mapping = {
    'Married': 'Relationship',
    'Together': 'Relationship',
    'Single': 'No Relationship',
    'Widow': 'No Relationship',
    'Alone': 'No Relationship',
    'Divorced': 'No Relationship',
}

df['Marital_Status'] = df['Marital_Status'].replace(category_mapping)

In [ ]:
# create a barplot of the combined marital status feature
column_to_plot = 'Marital_Status'
category_counts = df[column_to_plot].value_counts()
sorted_categories = category_counts.sort_values(ascending=False).index

plt.figure(figsize=(8, 6))
sns.countplot(x=column_to_plot, data=df, order=sorted_categories)
plt.title(f'Barplot of {column_to_plot}', fontsize=16)
plt.xlabel(column_to_plot)
plt.ylabel('Count')
plt.show()

In order to reduce the number of dimensions further, we could combine the "AcceptedCmp..." and "Response" columns by creating a new column that represents whether a customer is susceptible for marketing campaigns or not.

In [ ]:
# create combined column and drop original ones
campaign_columns = ['AcceptedCmp1','AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'Response']
df['CampaignAccepted'] = df[campaign_columns].any(axis=1)
df = df.drop(campaign_columns, axis=1)

In [ ]:
df.info()

After all these steps we have reduced our dataset down to only 16 columns.

### 4.2.4 One-Hot Encoding

Now that we have inspected our categorical variables we need to preprocess them in order to use them in our machine learning models. One popular method is called One-hot encoding and is a technique used in machine learning to convert categorical data into a numerical format. Each category in the original feature is transformed into a new binary feature, where 1 represents the presence of the category and 0 represents the absence. This way we preserve all the information in our dataset by making each category a seperate feature.

In [ ]:
# apply one-hot encoding to our categorical columns
df_encoded = pd.get_dummies(df, columns=['Education', 'Marital_Status', 'CampaignAccepted'])
df_encoded

One major disadvantage of this technique is the increasing number of dimensions in our dataset. After the encoding step we now have 22 features. The encoding results in a more sparse dataset, which might negatively impact the performance of some algorithms and increase computational requirements.

Thinking of clustering this means we get very high dimensional sparse spaces where our algorithm has to find points that are close together. Espacially with distance based algorithms like K-Means this can cause severe problems. Also, the features in high dimensions tend to be redundant and correlated with each other which is why models are likely to overfit as a consequence. For example, K-Means clustering is very vulnerable to this curse of dimensionality. One common approach to take care of this is to perform a Principal Component Analysis (PCA) upfront.

### 4.2.5 PCA

Principal Component Analysis is a commonly used technique for dimensionality reduction on large datasets. It allows us to transform a dataset with a large number of features into a lower-dimensional space while still preserving the most important patterns by finding the components that maximize the explained variance. Therefore, the resulting set of transformed features are called principal components. They are uncorrelated by nature since principal components are orthogonal to each other and ranked by their explained variance.

In [ ]:
n_components_range = range(len(df_encoded.columns))

# standardize data before PCA
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_encoded)

# PCA for each number of components
explained_variance = []
cumulative_variance = []
for n_components in n_components_range:
    pca = PCA(n_components = n_components)
    pca.fit(scaled_data)
    explained_variance.append(np.sum(pca.explained_variance_ratio_))
    cumulative_variance.append(np.sum(pca.explained_variance_ratio_))
    
# plot the cumulative explained variance
plt.figure(figsize=(10, 6))
plt.plot(n_components_range, cumulative_variance, marker='x')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Cumulative Explained Variance vs. Number of Components')
plt.xticks(n_components_range)
plt.grid()

# example: add horizontal line at 60% explained variance
explained_variance_60percent = 0.60
n_components_60percent = np.argmax(np.array(cumulative_variance) >= explained_variance_60percent)
plt.axhline(y=explained_variance_60percent, color='green', linestyle='--', label='60% Explained Variance')
plt.axvline(x=n_components_60percent, color='green', linestyle='--')

# example: add horizontal line at 95% explained variance
explained_variance_95percent = 0.95
n_components_95percent = np.argmax(np.array(cumulative_variance) >= explained_variance_95percent)
plt.axhline(y=explained_variance_95percent, color='red', linestyle='--', label='95% Explained Variance')
plt.axvline(x=n_components_95percent, color='red', linestyle='--')

plt.legend()
plt.show()

Selecting the number of components is of course a trade-off between reducing dimensionality and retaining as much information as possible. Higher numbers of components might capture more variance but can also lead to overfitting. On the other hand, too few components may result in a significant loss of information.

The first thing we notice from the above plot is that we do already capture all of the variance using only 20 components instead of 22. By defining a certain amount of variance that must be captured we can reduce the number of components even further. For example, looking at the plot we obtain 17 components for defining a variance threshold of 95% and only 6 components for 60% explained variance.

In [ ]:
# returns the principal components needed to achieve a certain explained variance
def GetPCAFeatures(variance_threshold):
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df_encoded)
    pca = PCA(n_components=None)
    pca.fit(scaled_data)
    
    cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(cumulative_variance >= variance_threshold) + 1
    pca = PCA(n_components=n_components)
    pca.fit(scaled_data)
    principal_components = pca.transform(scaled_data)

    # create a dataframe containing the reduced feature set
    df_pca = pd.DataFrame(principal_components, columns=[f'PC{i+1}' for i in range(n_components)])
    
    return df_pca

For easier comparability and visualization we will work with only 3 principal components which reflects an explained variability of around 39%. Let's see if this is enough to form meaningful clusters.

In [ ]:
df_pca = GetPCAFeatures(0.39)
df_pca.head()

# 5. Model Building

During this section we want to build three different unsupervised clustering models, namely K-Means, DBSCAN, and Hierarchical Clustering, and compare their results both graphically and using different evaluation metrics. The metrics to be considered are:

* Silhouette
* Davies-Bouldin Index

Both are commonly used in the context of clustering. 'Silhouette' quantifies how well-separated the clusters are and measures their separation of data points within clusters. The score ranges from -1 to 1. The other evaluation metric called 'Davies-Bouldin Index' measures the average similarity between each cluster and its most similar cluster, relative to the average dissimilarity between the clusters. The lower the value the better the clusters are defined.

For hyperparameter tuning we will use grid search. This technique searches through a predefined set of hyperparameter values and creates a grid of all possible value combinations. Then it evaluates each combination using one of the mentioned evaluation metrics above.

The business question we want to answer using the resulting cluster is the following: Can we devide customers into certain distinguishable clusters according to their income and money spent on our products? Answering this question can certainly help to identify the most receptive customer segment and focus our marketing efforts only on that particular group. We can also think of strategies to better involve customers currently spending less money so that we don't loose them completely.

## 5.1 K-Means

As already mentioned, K-Means is an unsupervised algorithm for clustering data points into distinct groups. It is a so-called distance-based algorithm since it tries to minimize the sum of squared distances between data points and their assigned cluster centers (= centroids). This optimization process iteratively updates the cluster assignments and the centroids to find the best solution. Each step aims to minimize the within-cluster variance and maximize the separation between clusters. Upfront, you have to specify the number of desired clusters as a hyperparameter.

The algorithm typically suffers from the "curse of dimensionality" which means that a high-dimensional feature space (= many features) causes the algorithm to not find the best clusters. In high-dimensional spaces the data points become very sparse which means that the distance between the points increases and it becomes much more difficult to find meaningful clusters.

The fact that K-Means may not be the ideal choice for high dimensions is also reflected in the formula for time complexity as follows: O(t * K * n * d)

* t: number of iterations until clusters are formed
* K: number of specified clusters
* n: number of data points
* d: number of dimensions (= features)

This means time complexity increases linearly with the number of features and also datapoints which both can be quite high numbers. Typically, the number of clusters is a smaller number and therefore not having a huge impact. The number of iterations needed until convergence on the other hand plays a more important role since it heavily depends on the initial placement of the centroids which can be affected by another hyperparameter.

With all this information we can conclude that this algorithm could be suitable for our data if we do not take too many features into the model. Therefore we will work with a smaller amount of explained variance in the PCA model to avoid the curse of dimensionality. Besides that, K-Means could also act as kind of a baseline model to compare with more sophisticated models like DBSCAN which we will explore later.

### 5.1.1 Basic model

In [ ]:
# get a fresh copy of the PCA dataset
df_pca_kmeans = df_pca.copy(True)

In [ ]:
# try out different number of clusters
num_clusters_range = range(2,10)

# calculate the inertia (= within-cluster sum of squares) for each cluster
# this is a measure for how well the algorithm performs
# the lower the inertia the better the clusters are formed
inertia = []
for num_clusters in num_clusters_range:
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(df_pca_kmeans)
    inertia.append(kmeans.inertia_)

# plot elbow curve
plt.figure(figsize=(8, 6))
plt.plot(num_clusters_range, inertia, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.title('K-Means Clustering')
plt.grid()
plt.show()

The plot shows the inertia (= within cluster sum-of-squares) for each K (= number of clusters). We can see that the inertia becomes smaller for higher number of clusters. We could say that we should choose the highest possible value for K then. But of course clusters become less meaningful the more there are. Therefore, we have to choose a K big enough where we still get meaningful clusters. A popular technique to do so is called the Elbow Method where we try to find the K in the graph where the decline in inertia becomes significantly smaller. From the plot above we see that it is the case for K = 4.

Let's repeat the K-Means for this number of clusters and store the resulting cluster assignment in our dataset.

In [ ]:
# Perform K-Means for optimal number of clusters acoording to Elbow Method
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans.fit(df_pca_kmeans)
df_pca_kmeans['KMeans_Basic'] = kmeans.labels_
df_encoded['KMeans_Basic'] = kmeans.labels_

In [ ]:
# plot the resulting clusters
fig=plt.figure(figsize=(12,8))
ax=plt.subplot(111, projection='3d')
ax.scatter(df_pca_kmeans['PC1'], df_pca_kmeans['PC2'], df_pca_kmeans['PC3'], s=40, c=df_pca_kmeans['KMeans_Basic'], marker='o', edgecolor='black')
ax.set_title('Plot of the Clusters for K-Means')
plt.show()

We can clearly see the different clusters formed in the 3D space represented by the 3 most important principal components. Now we will look at the original feature space and plot the total amount spent vs. the income to see if we can identify distinct groups of customers.

In [ ]:
# plot total amount spent vs. income
sns.scatterplot(data=df_encoded, x='Income', y='TotalSpent', hue='KMeans_Basic')
plt.title('K-Means Clusters based on Income and Spending')
plt.legend()
plt.show()

The plot indicates that there are indeed different groups of customers. There seem to be the one with really high income and larger amounts spent as well as those with medium income and medium amount spent. For lower income and money spent there is some overlap between two different clusters. We will see if this can be improved doing hyperparameter optimization. 

### 5.1.2 Hyperparameter Optimization

Besides the inital value for K we can set an initial state that impacts how fast the algorithm converges. We will try to find the best pair of parameters using a hyperparameter optimization. As already mentioned, we try out two different evaluation metrics to compare with each other. One is called 'Silhouette' and the other is called 'Davies-Bouldin Index'.

In [ ]:
# get a fresh copy of the PCA dataset
df_pca_kmeans = df_pca.copy(True)

In [ ]:
def KMeansHyper(scoring, columnName):   
    
    # define the parameter grid
    param_grid = {
        'n_clusters': range(3,5), # number of clusters
        'n_init': [10, 20, 30, 40],  # number of times K-Means algorithm runs with different centroid seeds
        'init': ['k-means++', 'random'],  # different centroid initialization methods
    }

    # perform grid search
    kmeans = KMeans()
    grid_search = GridSearchCV(kmeans, param_grid, scoring=scoring)
    grid_search.fit(df_pca_kmeans)

    # get best parameters
    best_k = grid_search.best_params_['n_clusters']
    best_n_init = grid_search.best_params_['n_init']
    best_init = grid_search.best_params_['init']
    best_kmeans = grid_search.best_estimator_
    print("Best K:", best_k)
    print("Iterations on centroid seeds:", best_n_init)
    print("Best Initialization Method:", best_init)

    # fit best model and store cluster results
    best_kmeans.fit(df_pca_kmeans)
    df_pca_kmeans[columnName] = best_kmeans.labels_
    df_encoded[columnName] = best_kmeans.labels_

In [ ]:
silhouette_scorer = make_scorer(silhouette_score)
KMeansHyper(silhouette_scorer, 'KMeans_Hyper_Sil')

In [ ]:
davies_bouldin_scorer = make_scorer(davies_bouldin_score, greater_is_better=False)
KMeansHyper(davies_bouldin_scorer, 'KMeans_Hyper_Bouldin')

Both evaluation metrics yield the same best hyperparameters for K-Means. Let's visualize the resulting clusters.

In [ ]:
# plot the resulting clusters
fig = plt.figure(figsize=(12, 8))
ax1 = plt.subplot(121, projection='3d')
ax2 = plt.subplot(122, projection='3d')

ax1.scatter(df_pca_kmeans['PC1'], df_pca_kmeans['PC2'], df_pca_kmeans['PC3'], s=40, c=df_pca_kmeans['KMeans_Hyper_Sil'], marker='o', edgecolor='black')
ax1.set_title('K-Means hyperparameter optimization (Silhouette)')

ax2.scatter(df_pca_kmeans['PC1'], df_pca_kmeans['PC2'], df_pca_kmeans['PC3'], s=40, c=df_pca_kmeans['KMeans_Hyper_Bouldin'], marker='o', edgecolor='black')
ax2.set_title('K-Means hyperparameter optimization (Davies-Bouldin)')

plt.show()

Again, we can clearly see the different clusters formed in the 3D space. Let's have a look at the original feature space.

In [ ]:
# plot total amount spent vs. income
def plot_kmeans_spentincome():
    fig = plt.figure(figsize=(14, 6))
    ax1 = plt.subplot(121) 
    ax2 = plt.subplot(122) 

    sns.scatterplot(data=df_encoded, x='Income', y='TotalSpent', hue='KMeans_Hyper_Sil', ax=ax1)
    ax1.set_title('K-Means hyperparameter optimization (Silhouette)')

    sns.scatterplot(data=df_encoded, x='Income', y='TotalSpent', hue='KMeans_Hyper_Bouldin', ax=ax2)
    ax2.set_title('K-Means hyperparameter optimization (Davies-Bouldin)')

    plt.show()

In [ ]:
plot_kmeans_spentincome()

We see that both evaluation metrics yield (nearly) the same results if we ignore the different color assignment to the clusters. We can easily recognize the 3 three clusters. In contrast to the intermediate result yielded by the basic model we can now clearly identify the clusters.

Let's see if we can reproduce or even improve the results using DBSCAN instead of K-Means.

## 5.2 DBSCAN

The term DBSCAN stands for Density-Based Spatial Clustering of Applications with Noise (DBSCAN) and as the name suggests it is a density-based clustering algorithm. It can be used to discover clusters of data points in a dataset but unlike K-Means, it does not require specifying the number of clusters upfront. Instead, the clusters are defined based on the density of data points in the feature space.

In contrast to K-Means this algorithm does not suffer as much from the "curse of dimensionality" since the optimization method does not purely rely on distanced based calulcations. The time complexity of DBSCAN is put together by several factors, also including the number of data points (n) and number of dimensions (= features) of the data (d). In contrast to K-Means we have no K (= number of clusters) or t (= number of iterations) but rather other parameters called Epsilon and MinPts.

Epsilon is defined as a threshold signaling two points are neighbors if the distance between the two points is below that threshold. MinPts on the other hand represents the minimum number of points needed to be called a dense cluster (using Epsilon to identify neighbors).

The worst-case time complexity of the DBSCAN algorithm is O(n^2) which can be the case for sparse datasets and smaller values of MinPts since this results in more time-intensive neighborhood searches. The average time complexity of DBSCAN is approximately O(n * log n), primarily due to the use of a so-called spatial index to efficiently find neighboring points. However, in cases of dense datasets with a large MinPts value, the complexity can also reach up to O(n^2).

This means time complexity for DBSCAN heavely relies on the number of data points and also on the choice of parameters Epsilon and MinPts. Considering this information we can say that this algorithm could probably be a better choice for our data.

### 5.2.1 Basic model

In [ ]:
# get a fresh copy of the PCA dataset
df_pca_dbscan = df_pca.copy(True)

In [ ]:
# create a DBSCAN model with arbitrary values for its parameters
dbscan = DBSCAN(eps=0.5, min_samples=20)
cluster_labels = dbscan.fit_predict(df_pca_dbscan)
df_pca_dbscan['DBSCAN_Basic'] = cluster_labels
df_encoded['DBSCAN_Basic'] = cluster_labels

In [ ]:
# plot the resulting clusters
fig=plt.figure(figsize=(12,8))
ax=plt.subplot(111, projection='3d')
ax.scatter(df_pca_dbscan['PC1'], df_pca_dbscan['PC2'], df_pca_dbscan['PC3'], s=40, c=df_pca_dbscan['DBSCAN_Basic'], marker='o', edgecolor='black')
ax.set_title('Plot of the Clusters for DBSCAN')
plt.show()

In [ ]:
sns.scatterplot(data=df_encoded, x='Income', y='TotalSpent', hue='DBSCAN_Basic')
plt.title('DBSCAN Clusters based on Income and Spending')
plt.legend()
plt.show()

We obtain 6 different clusters that do not seem to be very distinct which can be easily seen in both plot above. This makes it quite hard to draw any conclusions at this point. We can try to apply a hyperparameter optimization in the next step to make sure we were not just unlucky with our initial choice of parameters.  

### 5.2.2 Hyperparameter Optimization

In contrast to K-Means we do not have to specify the number if clusters in advance. Instead we will vary the two parameters 'eps' and 'min_samples'. The first one sets a threshold for the maximum distance between two samples for one to be considered as part of the neighborhood. The second one sets the minimum number of samples in a neighborhood for a point to be considered as a core point. Again we will experiment with the two scorers 'Silhouette' and 'Davies-Bouldin Index'.

In [ ]:
# get a fresh copy of the PCA dataset
df_pca_dbscan = df_pca.copy(True)

In [ ]:
def DBSCANHyper(scoring, columnName):   
    
    # define the parameter grid
    param_grid = {
        'eps': [1, 10, 1],    # maximum distance between two samples for one to be considered as part of the neighborhood of the other
        'min_samples': [5, 10, 15, 20],          # minimum number of samples in a neighborhood for a point to be considered as a core point
    }

    # perform grid search
    dbscan = DBSCAN()
    grid_search = GridSearchCV(dbscan, param_grid, scoring=scoring)
    grid_search.fit(df_pca_dbscan)

    # get best parameters
    best_dbscan = grid_search.best_estimator_
    print("Best eps:", grid_search.best_params_['eps'])
    print("Best min_samples:", grid_search.best_params_['min_samples'])

    # fit best model and store cluster results
    best_dbscan.fit(df_pca_dbscan)
    df_pca_dbscan[columnName] = best_dbscan.labels_
    df_encoded[columnName] = best_dbscan.labels_

In [ ]:
silhouette_scorer = make_scorer(silhouette_score)
DBSCANHyper(silhouette_scorer, 'DBSCAN_Hyper_Sil')

In [ ]:
davies_bouldin_scorer = make_scorer(davies_bouldin_score, greater_is_better=False)
DBSCANHyper(davies_bouldin_scorer, 'DBSCAN_Hyper_Bouldin')

Again, both evaluation metrics yield the same best hyperparameters. Let's visualize the resulting clusters.

In [ ]:
# plot the resulting clusters
fig = plt.figure(figsize=(12, 8))
ax1 = plt.subplot(121, projection='3d')
ax2 = plt.subplot(122, projection='3d')

ax1.scatter(df_pca_dbscan['PC1'], df_pca_dbscan['PC2'], df_pca_dbscan['PC3'], s=40, c=df_pca_dbscan['DBSCAN_Hyper_Sil'], marker='o', edgecolor='black')
ax1.set_title('DBSCAN hyperparameter optimization (Silhouette)')

ax2.scatter(df_pca_dbscan['PC1'], df_pca_dbscan['PC2'], df_pca_dbscan['PC3'], s=40, c=df_pca_dbscan['DBSCAN_Hyper_Bouldin'], marker='o', edgecolor='black')
ax2.set_title('DBSCAN hyperparameter optimization (Davies-Bouldin)')

plt.show()

In [ ]:
# plot total amount spent vs. income
def plot_dbscan_spentincome():
    fig = plt.figure(figsize=(14, 6))
    ax1 = plt.subplot(121) 
    ax2 = plt.subplot(122) 

    sns.scatterplot(data=df_encoded, x='Income', y='TotalSpent', hue='DBSCAN_Hyper_Sil', ax=ax1)
    ax1.set_title('DBSCAN hyperparameter optimization (Silhouette)')

    sns.scatterplot(data=df_encoded, x='Income', y='TotalSpent', hue='DBSCAN_Hyper_Bouldin', ax=ax2)
    ax2.set_title('DBSCAN hyperparameter optimization (Davies-Bouldin)')

    plt.show()

In [ ]:
plot_dbscan_spentincome()

This time we do not obtain any meaningful results. Our model identified two clusters but one of them contains nearly all data points and the other one only a few outliers. This could have several reasons which we will discuss in more detail in the Results and Evaluation chapter.

Let's see if we can achieve better results again using a hierarchical clustering approach.

## 5.3 Hierarchical Clustering

Agglomerative Clustering is a bottom-up hierarchical clustering algorithm that initially begins with each data point as its own cluster. Then it iteratively merges the closest clusters based on a distance metric until a termination criterion is met. The time complexity is O(n^3), with n as the number of data points. The repeated computation of pairwise distances between clusters during each iteration is the reason for this complexity and makes it much more computationally expensive than K-Means and DBSCAN. However, for our case this should not be a problem because we are only working with roughly over 2000 data points.

Since this algorithm can handle different cluster shapes and sizes it seems appropriate for most datasets and also the one we are using. Our data might not be hierarchical by nature but this algorithm could help identify nested clusters.

### 5.3.1 Basic model

Similar to K-Means we have to specify the value for the number of clusters upfornt. In order to evaluate resulting clusters we choose the Calinski-Harabasz Index as we did with the inertia for K-Means. Also known as the Variance Ratio Criterion (VRC), this metric measures the ratio of between-cluster variance to within-cluster variance and therefore provides a measure of cluster separation. The higher the index value, the better the clustering solution. In other words we are looking for a peak in the plot instead of using the Elbow Method.

In [ ]:
# get a fresh copy of the PCA dataset
df_pca_agglo = df_pca.copy(True)

In [ ]:
# try out different number of clusters
num_clusters_range = range(3,10)

# calculate the calinski_harabasz_score as a measure of how well the clusters are formed
calinski_scores = []
for num_clusters in num_clusters_range:
    agglo = AgglomerativeClustering(n_clusters=num_clusters)
    cluster_labels = agglo.fit_predict(df_pca_agglo)
    calinski_scores.append(calinski_harabasz_score(df_pca_agglo, cluster_labels))

# plot the index curve
plt.figure(figsize=(8, 6))
plt.plot(num_clusters_range, calinski_scores, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Calinski-Harabasz Index')
plt.title('Agglomerative Clustering')
plt.grid()
plt.show()

We identify the peak to be at 6 clusters. This seems to be quite a lot. Let's visualize the result to see more.

In [ ]:
# create a Agglomerative clustering model with arbitrary values for its parameters
agglo = AgglomerativeClustering(n_clusters=6)
agglo_labels = agglo.fit_predict(df_pca_agglo)
df_pca_agglo['Agglo_Basic'] = agglo_labels
df_encoded['Agglo_Basic'] = agglo_labels

In [ ]:
# plot the resulting clusters
fig=plt.figure(figsize=(12,8))
ax=plt.subplot(111, projection='3d')
ax.scatter(df_pca_agglo['PC1'], df_pca_agglo['PC2'], df_pca_agglo['PC3'], s=40, c=df_pca_agglo['Agglo_Basic'], marker='o', edgecolor='black')
ax.set_title('Plot of the Clusters for Agglomerative Clustering')
plt.show()

In [ ]:
sns.scatterplot(data=df_encoded, x='Income', y='TotalSpent', hue='Agglo_Basic')
plt.title('Agglomerative Clusters based on Income and Spending')
plt.legend()
plt.show()

Despite having quite clearly seperated clusters in the first plot we do not obtain a meaningful clustering for our specific Income vs. Spending plot. Maybe we can improve this by changing the hyperparameters.

### 5.3.2 Hyperparameter Optimization

Even though we have already found a good value for K, we let the hyperparameter search find the best value in combination with other parameters. Those are 'affinity' and 'linkage'. The first one refers to the distance metric used and the second one to the linkage criterion that decides whether two centroids are merged together or not.

In [ ]:
# get a fresh copy of the PCA dataset
df_pca_agglo = df_pca.copy(True)

In [ ]:
def AggloHyper(scoring, columnName):   
    
    # define the parameter grid
    param_grid = {
        'n_clusters': [3, 4, 5, 6],  # number of clusters
        'affinity': ['euclidean', 'l1', 'l2', 'manhattan'],  # distance metric
        'linkage': ['ward', 'complete', 'average', 'single']  # linkage criterion
    }

    # perform grid search
    agglomerative = AgglomerativeClustering()
    grid_search = GridSearchCV(agglomerative, param_grid, scoring=scoring)
    grid_search.fit(df_pca_agglo)

    # get best parameters
    best_agglomerative = grid_search.best_estimator_
    print("Best n_clusters:", grid_search.best_params_['n_clusters'])
    print("Best affinity:", grid_search.best_params_['affinity'])
    print("Best linkage:", grid_search.best_params_['linkage'])

    # fit best model and store cluster results
    best_agglomerative.fit(df_pca_agglo)
    df_pca_agglo[columnName] = best_agglomerative.labels_
    df_encoded[columnName] = best_agglomerative.labels_

In [ ]:
silhouette_scorer = make_scorer(silhouette_score)
AggloHyper(silhouette_scorer, 'Agglo_Hyper_Sil')

In [ ]:
davies_bouldin_scorer = make_scorer(davies_bouldin_score, greater_is_better=False)
AggloHyper(davies_bouldin_scorer, 'Agglo_Hyper_Bouldin')

Again, both evaluation metrics yield the same best hyperparameters. Let's visualize the resulting clusters.

In [ ]:
# plot the resulting clusters
fig = plt.figure(figsize=(12, 8))
ax1 = plt.subplot(121, projection='3d')
ax2 = plt.subplot(122, projection='3d')

ax1.scatter(df_pca_agglo['PC1'], df_pca_agglo['PC2'], df_pca_agglo['PC3'], s=40, c=df_pca_agglo['Agglo_Hyper_Sil'], marker='o', edgecolor='black')
ax1.set_title('Agglomerative hyperparameter optimization (Silhouette)')

ax2.scatter(df_pca_agglo['PC1'], df_pca_agglo['PC2'], df_pca_agglo['PC3'], s=40, c=df_pca_agglo['Agglo_Hyper_Bouldin'], marker='o', edgecolor='black')
ax2.set_title('Agglomerative hyperparameter optimization (Davies-Bouldin)')

plt.show()

In [ ]:
# plot total amount spent vs. income
def plot_agglo_spentincome():
    fig = plt.figure(figsize=(14, 6))
    ax1 = plt.subplot(121) 
    ax2 = plt.subplot(122) 

    sns.scatterplot(data=df_encoded, x='Income', y='TotalSpent', hue='Agglo_Hyper_Sil', ax=ax1)
    ax1.set_title('Agglormerative hyperparameter optimization (Silhouette)')

    sns.scatterplot(data=df_encoded, x='Income', y='TotalSpent', hue='Agglo_Hyper_Bouldin', ax=ax2)
    ax2.set_title('Agglormerative hyperparameter optimization (Davies-Bouldin)')

    plt.show()

In [ ]:
plot_agglo_spentincome()

Interestingly, the hyperparameter search yields a K = 3 instead of 6 which also seemed to be a reasonable choice considering the plot for the Calinski-Harabasz Index at the beginning of this section. Now, the change in K could be affected by the change of the other parameters during the tuning process. Both plots show one cluster that is clearly seperated whereas the other two are more overlapped which makes it a bit hard to perform a final and clear interpretation of the results.

# 6. Results and Evaluation

We worked with three different models, namely K-Means, DBSCAN and Hierarchical respectively Agglomerative Clustering. For each of these models we experimented with a basic model where no hyperparameters were tuned to get a feeling of how good this model might work. Then we performed hyperparameter tuning using grid search and also compared two different evaluation metrics (Silhouette and Davies-Bouldin). We already explained at the beginning of the model building section how grid search as an tuning technique works.

For each of our models we visualized the clusters in 3D space and also had in mind our business related question which asked if we can devide customers into certain distinguishable clusters according to their income and money spent on our products. In this section we want to summarize the results from the model building section and try to answer this question. We also want to derive which model works best for our dataset.

## 6.1 Visualizations

For easier comparison we will plot the resulting spent vs. income plots from our hyperparameter tuning parts again.

In [ ]:
plot_kmeans_spentincome()

In [ ]:
plot_dbscan_spentincome()

In [ ]:
plot_agglo_spentincome()

## 6.2 Interpretation

Let's look at the evaluation metrics first. For each of our plot pairs we cannot detect any noticable difference when it comes to the clustering result considering the Silhouette and the Davies-Bouldin metric. Therefore we will only focus on the Silouette plots for our further conlusions.

Talking about the quality of the results it is clear that the DBSCAN clustering performed really bad in answering our business question. We cannot derive any meaningful customer groups from the plot since most the data points belong the same cluster. One reason for this bad result could be that the algorithm is considering almost all data points as noise points, resulting in a single cluster containing the majority of the data. In turn, this could be the result of a epsilon value that is too small. But since our hyperparameter search proposed this value as the best it could be that our data is not ideally suited for this type of algorithm. It could be that the data structure is not optimal or that we should improve our preprocessing steps, e.g. by removing more outliers from other features or taking more principal components into account for the model building itself.

Agglomerative clustering performed much better than DBSCAN but not as good a K-Means. The latter allows for an interpretation based on the detected number of clearly seperated clusters. The clusters we can name are:

* Customers with high income and high spending
* Customers with intermediate income and intermediate spending
* Customers with low to intermediate income and less spending

What is not reflected by the clusters but what we can notice (ignoring outliers) is that the income range in the last cluster is the largest. This means there are indeed people with at least intermediate income that do not spend a lot on our products (right side of the red cluster in the first pair of plots). By improving our models so that we can get those to be reflected by an extra cluster we could precisely target certain marketing strategies to these customers. This might be very lucrative since there is a lot of room for spending more money on our products. Additionally, the remaining part of the group with lower income could be the target of specific marketing strategies that focus on discounts making more products affordable to people with lower income.

Exploring the dataset and clustering further we could try to link this information with the original information about how much was spent on certain products. This way we could target the discounts specifically on those products that are actually of interest.

# 7. Conclusion

As a final step in this project let's talk about what did work out well and where things could be improved.

## 7.1 Learnings and Takeaways

Going through a complete machine learning project highlights the crucial role of preparation steps such as data cleaning and preprocessing before model building. For example dealing with missing values can be really important when the values belong to one of the features of main interest (Income feature in this case). Techniques like one-hot encoding to deal with categorical data often causes the created features to be correlated.Therefore, subsequent preprocessing steps like PCA that exhibit no correlation in the resulting dataset cannot be ignored. It's also important to recognize that not all models are suitable for refering to the same dataset, and careful consideration is needed when preprocessing the data or selecting the best number of principal components after PCA.

## 7.2 What did not work

As already stated the DBSCAN did not work very well. The unsatisfactory outcome may have resulted from the algorithm classifying nearly all data points as noise. This might be a consequence of using a small epsilon value during the hyperparameter search, suggesting that the algorithm may not be an ideal fit. It is possible that the data's structure is suboptimal, or our preprocessing step requires improvement. For instance, addressing outliers in other features or considering additional principal components during model building might yield better results.

Additionally, it would have been nice to have a fourth cluster showing the customers with intermediate income and lower spending rate. Especially, these people have a huge potential in terms of buying power.

## 7.3 Possible Improvements

To improve the clustering results, we should focus on more detailed outlier handling to reduce noise, since noise can negatively impact some clustering algorithms (probably the DBSCAN algorithm). It is also be beneficial to avoid combining too many features to prevent information loss during the clustering process. Besides that, we could experiment with different numbers of Principal Components including a higher explained variance to identify the optimal representation for the data. Of course, trying out other clustering techniques such as Gaussian mixture models can help us find the most suitable algorithm for our specific problem.